<a href="https://colab.research.google.com/github/Suryapv26/grokking/blob/main/ANN_Churn_A_Z.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import libraries

In [1]:

import numpy as np
import pandas as pd
import tensorflow as tf

Open dataset

In [2]:

dataset = pd.read_csv('Churn_Modelling.csv')

print(type(dataset))

<class 'pandas.core.frame.DataFrame'>


Get unique values of cat col

In [3]:
print(dataset['Gender'].unique())
print(dataset['Geography'].unique())

['Female' 'Male']
['France' 'Spain' 'Germany']


Getting the count of values of geography colum

In [4]:
dataset['Geography'].value_counts()

France     5014
Germany    2509
Spain      2477
Name: Geography, dtype: int64

One hot encoding of cat columns.
Redundant columns are dropped

In [19]:

one_hot_encoded_data = pd.get_dummies(dataset, columns = ['Geography', 'Gender'])
print(one_hot_encoded_data)
df = one_hot_encoded_data.drop(['RowNumber','CustomerId','Surname'],axis = 1)

df = df.drop(['Geography_Spain','Gender_Female'],axis = 1)

      RowNumber  CustomerId    Surname  CreditScore  Age  Tenure    Balance  \
0             1    15634602   Hargrave          619   42       2       0.00   
1             2    15647311       Hill          608   41       1   83807.86   
2             3    15619304       Onio          502   42       8  159660.80   
3             4    15701354       Boni          699   39       1       0.00   
4             5    15737888   Mitchell          850   43       2  125510.82   
...         ...         ...        ...          ...  ...     ...        ...   
9995       9996    15606229   Obijiaku          771   39       5       0.00   
9996       9997    15569892  Johnstone          516   35      10   57369.61   
9997       9998    15584532        Liu          709   36       7       0.00   
9998       9999    15682355  Sabbatini          772   42       3   75075.31   
9999      10000    15628319     Walker          792   28       4  130142.79   

      NumOfProducts  HasCrCard  IsActiveMember  Est

In [20]:
print(df)

      CreditScore  Age  Tenure    Balance  NumOfProducts  HasCrCard  \
0             619   42       2       0.00              1          1   
1             608   41       1   83807.86              1          0   
2             502   42       8  159660.80              3          1   
3             699   39       1       0.00              2          0   
4             850   43       2  125510.82              1          1   
...           ...  ...     ...        ...            ...        ...   
9995          771   39       5       0.00              2          1   
9996          516   35      10   57369.61              1          1   
9997          709   36       7       0.00              1          0   
9998          772   42       3   75075.31              2          1   
9999          792   28       4  130142.79              1          1   

      IsActiveMember  EstimatedSalary  Exited  Geography_France  \
0                  1        101348.88       1                 1   
1            

get column names as list

In [21]:
list(df.columns.values)


['CreditScore',
 'Age',
 'Tenure',
 'Balance',
 'NumOfProducts',
 'HasCrCard',
 'IsActiveMember',
 'EstimatedSalary',
 'Exited',
 'Geography_France',
 'Geography_Germany',
 'Gender_Male']

rearrange the columns to bring dependent variable column to the end so that splitting of test and train set would be easier

In [22]:
df = df[['CreditScore',
 'Age',
 'Tenure',
 'Balance',
 'NumOfProducts',
 'HasCrCard',
 'IsActiveMember',
 'EstimatedSalary',
  'Geography_France',
 'Geography_Germany',
 'Gender_Male','Exited']]


In [23]:
df

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Gender_Male,Exited
0,619,42,2,0.00,1,1,1,101348.88,1,0,0,1
1,608,41,1,83807.86,1,0,1,112542.58,0,0,0,0
2,502,42,8,159660.80,3,1,0,113931.57,1,0,0,1
3,699,39,1,0.00,2,0,0,93826.63,1,0,0,0
4,850,43,2,125510.82,1,1,1,79084.10,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,39,5,0.00,2,1,0,96270.64,1,0,1,0
9996,516,35,10,57369.61,1,1,1,101699.77,1,0,1,0
9997,709,36,7,0.00,1,0,1,42085.58,1,0,0,1
9998,772,42,3,75075.31,2,1,0,92888.52,0,1,1,1


In [24]:
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

In [34]:
X.shape

(10000, 11)

In [25]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [26]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [27]:
ann = tf.keras.models.Sequential()

In [29]:
#add fully connected layer units--> #neurons no rule of thumb exist
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))
#binary classifier hence 1 neuron in o/p layer and we need probabilities hence using sigmoid
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

#since we have binary classifier use binary cross entropy. if categorical use 
#categorical cross entropy with activation = softmax
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
250/250 [==============================] - 1s 2ms/step - loss: 0.6050 - accuracy: 0.7256
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4955 - accuracy: 0.7960
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4577 - accuracy: 0.7960
Epoch 4/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4342 - accuracy: 0.8006
Epoch 5/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4261 - accuracy: 0.8084
Epoch 6/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4221 - accuracy: 0.8160
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4197 - accuracy: 0.8181
Epoch 8/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4173 - accuracy: 0.8196
Epoch 9/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4153 - accuracy: 0.8227
Epoch 10/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4137 - accura

In [ ]:
'CreditScore',
 'Age',
 'Tenure',
 'Balance',
 'NumOfProducts',
 'HasCrCard',
 'IsActiveMember',
 'EstimatedSalary',
  'Geography_France',
 'Geography_Germany',
 'Gender_Male','Exited'

Use our ANN model to predict if the customer with the following informations will leave the bank:

Geography: France

Credit Score: 600

Gender: Male

Age: 40 years old

Tenure: 3 years

Balance: $ 60000

Number of Products: 2

Does this customer have a credit card ? Yes

Is this customer an Active Member: Yes

Estimated Salary: $ 50000

So, should we say goodbye to that customer ?

In [35]:
print(ann.predict(sc.transform([[600,40,3,60000,2,1,1,50000,1,0,0]])) > 0.5)

1/1 [==============================] - 0s 20ms/step
[[False]]


In [32]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

63/63 [==============================] - 0s 1ms/step
[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


In [33]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1528   67]
 [ 204  201]]


0.8645

In [18]:
dataset1 = pd.read_csv('Churn_Modelling.csv')

X1 = dataset1.iloc[:, 3:-1].values
y1 = dataset1.iloc[:, -1].values
print(X1.shape)
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X1[:, 2] = le.fit_transform(X1[:, 2])

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X1 = np.array(ct.fit_transform(X1))

(10000, 10)


In [7]:
print(X1.shape)

(10000, 12)


In [8]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X1 = np.array(ct.fit_transform(X1))

In [9]:
print(X1.shape)

(10000, 13)
